In [10]:
from collections import Counter

from pyparsing import WordStart
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
from io import StringIO
import math
import gensim.downloader as api
import warnings
from gensim import W


class PreprocessData:
    def __init__(self,path,lang='english'): 
        self.dataset=pd.read_csv(path, sep=",")
        self.dataset=self.dataset[["content","score"]]
        self.stopwords=stopwords.words(lang)
        self.preprocess()
    
    @staticmethod
    def remove_punctuation(text):
        '''a function for removing punctuation'''
        # replacing the punctuations with no space, 
        # which in effect deletes the punctuation marks 
        translator = str.maketrans('', '', string.punctuation)
        # return the text stripped of punctuation marks
        return text.translate(translator)

    #A function to remove the stopwords
    def remove_stopwords(self,text):
        text = [word.lower() for word in text.split() if word.lower() not in self.stopwords]
        # joining the list of words with space separator
        return " ".join(text)

    def preprocess(self):
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_punctuation)
        self.dataset.iloc[:,0] = self.dataset.iloc[:,0].apply(self.remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
path="/content/reviews.csv"
dataset=PreprocessData(path)

In [12]:
dataset.dataset.head()

,content,score
0,cannot open app anymore,1
1,begging refund app month nobody replying,1
2,costly premium version approx indian rupees 91...,1
3,used keep organized 2020 updates made mess thi...,1
4,dan birthday oct 28,1


In [13]:
wv = api.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [20]:
# from gensim.scripts.glove2word2vec import glove2word2vec

In [89]:
# import gensim.models

# # This should take approximately 3 minutes
# model = gensim.models.Word2Vec(sentences=sentences,min_count=1)

In [90]:
# print(list(model.wv.vocab))

In [86]:
class WordRepresentation:
    def __init__(self,dataset,wv):
        self.dataset=dataset
        self.glove={}
        self.word2vec={}
        self.compute_glove()
        # self.compute_word2vec()
    
    #One hot
    def compute_glove(self):
        if self.word2vec:
            #The case if we have the ponderations or the frequencies , we don't need to search the words again
            #This condition will reduce the time complexity a lot   
            for word in self.word2vec:
              if word in wv.vocab:
                self.glove[word]=wv.word_vec(word)
        else:
          for i in range(len(self.dataset)):
            document=self.dataset.iloc[i,0]
            word_list=document.split(" ")
            for word in word_list:
              if word in wv.vocab:
                self.glove[word]=wv.word_vec(word)

In [87]:
new_data=WordRepresentation(dataset.dataset.iloc[:20],wv)

In [88]:
new_data.glove

{'101': array([-0.59852  ,  0.64086  ,  0.93098  , -0.19479  ,  0.019493 ,
        -0.65966  ,  0.088026 , -0.95382  ,  0.0039027,  0.76247  ,
        -0.84725  , -0.40724  , -0.19367  ,  0.43216  , -0.45563  ,
        -0.94187  , -0.4718   ,  0.68029  , -1.1544   ,  0.51706  ,
         0.011988 ,  0.66893  ,  0.60671  , -0.20234  , -0.12696  ,
        -0.21745  , -0.075545 , -0.05793  , -0.12085  , -0.62517  ,
         1.6911   ,  0.16939  ,  0.48131  ,  0.36346  ,  0.8386   ,
        -0.59986  ,  0.80813  , -0.24383  ,  0.39558  ,  0.36722  ,
        -0.75517  , -0.54937  ,  1.2036   ,  0.36799  , -1.1024   ,
         1.2937   , -0.3816   , -0.10116  ,  0.8233   ,  0.13366  ],
       dtype=float32),
 '2020': array([ 0.38043  ,  0.6937   ,  1.6481   , -0.91584  , -0.27103  ,
        -0.62123  , -0.038582 , -1.3919   ,  0.49053  ,  0.5641   ,
         0.39801  , -0.58278  ,  0.31008  ,  0.53023  ,  0.80922  ,
         0.73065  ,  0.99547  ,  0.85719  , -0.61399  ,  0.17826  ,
         